# ReMKiT1D input generator - Poisson solver

This notebook creates the json input file for a poisson solver model.
The model being solved is:
$$
\nabla^2\phi = f
$$
Where $f$ is the charge distribution.

In [114]:
import numpy as np
import xarray as xr
import sys

import holoviews as hv
import matplotlib.pyplot as plt
import matplotlib as mpl
from holoviews import opts
import panel as pn

sys.path.append('../')
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
from RMK_support import RKWrapper ,Grid
import RMK_support.sk_normalization as skn

### Useful constants

In [115]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27  # atomic mass unit
ionMass = 2.014*amu  # deuterium mass
epsilon0 = 8.854188e-12  # vacuum permittivity
heavySpeciesMass = 2.014  # in amus

### Wrapper initialisation

In [116]:
rk = RKWrapper()

### Global parameters for IO files

In [117]:
rk.jsonFilepath = "./config.json"  # Default value
hdf5Filepath = "./RMKOutput/RMK_poisson/"
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

### MPI

In [118]:
numProcsX = 4  # Number of processes in x direction
numProcsH = 1  # Number of processes in harmonic
numProcs = numProcsX * numProcsH
haloWidth = 1  # Halo width in cells

rk.setMPIData(numProcsX, numProcsH, haloWidth)

### Normalisation

In [119]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

# for convenience
tempNorm = rk.normalization["eVTemperature"]
densNorm = rk.normalization["density"]

norms = skn.calculateNorms(tempNorm,densNorm,1.0)

timeNorm = norms["time"]
sigmaNorm = norms["crossSection"]
lengthNorm = norms["length"]
speedNorm = norms["speed"]

### Grid initialisation

In [120]:
dx0 = 2*0.27/4
dxN = 2*0.0125/4
Nx = 64*4
L = 10 # Length in meters
xGridWidths = L/Nx*np.ones(Nx)
dv0 = 0.05
dvN = 0.4
Nv = 80
vGridWidths = np.geomspace(dv0, dvN, Nv)
lMax = 0
gridObj = Grid(xGridWidths, vGridWidths, lMax, interpretXGridAsWidths=True,
               interpretVGridAsWidths=True, isLengthInMeters=True)

In [121]:
# Add the grid to the config file
rk.grid = gridObj

### Creating initial charge density configuration

In [122]:
rho = 0.05*np.exp(-(gridObj.xGrid-L/2)**2)

### Creating variables

In [123]:
rk.addVar("phi", isCommunicated=True)
rk.addVar("rho", rho, isCommunicated=True)

# Set scalar quantities 
rk.addVar("time",isScalar=True,isDerived=True)

### Creating Poisson equation model

In [124]:
# Adding the model tag to tag list
modelTag = "poisson"

# Initializing model
poissonModel = sc.CustomModel(modelTag=modelTag)

# Setting normalization constant calculation 
normConstPhi = sc.CustomNormConst(multConst=-1.0)
normConstF = sc.CustomNormConst(multConst=elCharge/epsilon0)

evolvedVar = "phi"

# Adding custom derivation for diffusion coefficient
rk.addCustomDerivation("diffusionDeriv",sc.simpleDerivation(1.0,[1.0]))

# Adding first laplacian term from poisson's eq.
laplacian = sc.GeneralMatrixTerm(evolvedVar,customNormConst=normConstPhi,stencilData=sc.diffusionStencil("diffusionDeriv",["phi"]))

poissonModel.addTerm("laplacian",laplacian)

# Adding charge distribution term
chargeDist = sc.GeneralMatrixTerm(evolvedVar,implicitVar="rho",customNormConst=normConstF,stencilData=sc.diagonalStencil())

poissonModel.addTerm("chargeDist",chargeDist)

rk.addModel(poissonModel.dict())

### Integrator options

In [125]:
integrator = sc.picardBDEIntegrator(absTol=10.0)

rk.addIntegrator("BE",integrator)

initialTimestep=0.1
rk.setIntegratorGlobalData(initialTimestep=initialTimestep)

bdeStep = sc.IntegrationStep("BE",defaultUpdateModelData=True)

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(8000)
rk.setFixedStepOutput(500)

rk.writeConfigFile()

### Analysis

In [126]:
numFiles = 16

loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)

loadedData

KeyError: "Unable to open object (object 'time' doesn't exist)"